In [1]:
%load_ext autoreload
%autoreload 1
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [2]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import timm
from timm import optim, scheduler
import torch
from torchvision import transforms as tfm
from sklearn import metrics as skmet
import matplotlib.pyplot as plt
import json
import transforms as my_transforms

%aimport dataset
from models import MultiTaskFrameClassifier
ImageData = dataset.ImageData

In [3]:
artifact_folder = '/zfs/wficai/pda/model_run_artifacts/20220818_multitask_224x224'

with open(artifact_folder + '/config.json', 'r') as f: 
    cfg = json.load(f)

# put all config variables in scope to avoid the need to laboriously index cfg
for k, v in cfg.items():
    v = f"'{v}'" if type(v)==str else v
    exec(f"{k}={v}")
del cfg

In [4]:
# optionally override settings
view_filter = ['pdaView', 'pdaRelatedView', 'nonPDAView']
mode_filter = ['2d', 'color', 'color_compare']
device = torch.device('cuda:1')  # you may need 'cuda:0'

In [5]:
tfms = my_transforms.ImageTransforms(res)
tfms_test = tfms.get_transforms(transforms['test'])

In [6]:
df_test = pd.read_csv(f'{artifact_folder}/{out_paths["test"]}')
d_test = ImageData(df_test, transforms = tfms_test, mode_filter = mode_filter, view_filter = view_filter)
dl_test = DataLoader(d_test, batch_size=bs_test, num_workers=num_workers)

print("Number of frames after filtering:", len(d_test.data))

Number of frames after filtering: 44961


In [25]:
# create model
encoder = timm.create_model(model, pretrained=pretrained, num_classes=1, in_chans=3, drop_rate=dropout)
clf = MultiTaskFrameClassifier(encoder).to(device)    
clf.load_state_dict(torch.load(f"{artifact_folder}/model_checkpoint.ckpt"))
clf.eval()
loss_function = MultiTaskFrameClassifier.multi_task_loss

target_ls = []
output_ls = []
study_ls = []
video_ls = []
view_ls = []
mode_ls = []
losses = []

for ix, batch in enumerate(dl_test):
    print(f"Batch {ix+1}", end = "\r")
    inputs = batch['img'].to(device)
    targets = {k: batch[k].to(device).type(torch.float32) for k in ['trg_type', 'trg_mode', 'trg_view']}
    
    target_ls.append(batch['trg_view'].numpy())
    view_ls.append(batch['trg_view'].numpy())
    mode_ls.append(batch['trg_mode'].numpy())
    study_ls += batch['study']
    video_ls += batch['video']

    with torch.no_grad():
        outputs = clf(inputs)
        output_ls.append(outputs)
        loss = loss_function(outputs, targets, weights)
        losses.append(loss)

# TODO: Pick up from here

In [13]:
df_results = pd.DataFrame(dict(
    study = study_ls,
    video = video_ls,
    predicted = np.concatenate(output_ls).squeeze(),
    target = np.concatenate(target_ls), 
    mode = np.concatenate(mode_ls),
    view = np.concatenate(view_ls)
))

df_results['mode'] = df_results['mode'].map(ImageData.inv_mode_map)
df_results.view = df_results.view.map(ImageData.inv_view_map)

df_results.predicted = 1 / (1 + np.exp(-df_results.predicted))

df_results.head(20)

ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
def compute_metrics(y_true, y_pred, thresh=0.5):
    mets = dict()
    
    y_pred_cls = (y_pred>thresh).astype(int)
    
    mets['num_samples'] = len(y_true)
    mets['roc_auc'] = skmet.roc_auc_score(y_true, y_pred)
    mets['average_precision'] = skmet.average_precision_score(y_true, y_pred)
    mets['accuracy'] = skmet.accuracy_score(y_true, y_pred_cls)
    mets['sensitivity'] = skmet.recall_score(y_true, y_pred_cls)
    mets['specificity'] = skmet.recall_score(y_true, y_pred_cls, pos_label=0)
    
    return mets

# Frame-level results

In [ ]:
compute_metrics(df_results.target, df_results.predicted)

In [ ]:
grouped_results = df_results.groupby(['view', 'mode']).apply(lambda dat: compute_metrics(dat.target, dat.predicted))
grouped_results = pd.DataFrame(grouped_results.tolist(), index=grouped_results.index)
grouped_results

# Clip-level results

### Avg confidence over frames

In [ ]:
df_results_clip_avg = df_results.groupby(['study', 'video', 'target', 'view', 'mode'], as_index=False).agg('mean')
display(df_results_clip_avg.head(10))

compute_metrics(df_results_clip_avg.target, df_results_clip_avg.predicted)

In [ ]:
grouped_results_clip_avg = df_results_clip_avg.\
    groupby(['view', 'mode']).\
    apply(lambda dat: compute_metrics(dat.target, dat.predicted, thresh=0.55))
grouped_results_clip_avg = pd.DataFrame(grouped_results_clip_avg.tolist(), index=grouped_results.index)
grouped_results_clip_avg

### Misses

In [ ]:
true = df_results_clip_avg.target
pred_cls = (df_results_clip_avg.predicted>0.5).astype(int)

df_results_clip_avg[true != pred_cls]

# Study-level results

In [ ]:
df_results_study_avg = df_results_clip_avg.groupby(['study', 'target'], as_index=False).agg('mean')
display(df_results_study_avg)

compute_metrics(df_results_study_avg.target, df_results_study_avg.predicted)